In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Read data from excel 
data = pd.read_excel('Dry_Bean_Dataset.xlsx')

X = data.drop(columns = ['Class'])
y = data['Class']

# Use mapping for the labels
# y = y.map({'SEKER': 0, 'BARBUNYA': 1, 'BOMBAY': 2, 'CALI': 3, 'HOROZ': 4, 'SIRA': 5, 'DERMASON': 6})

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)



# print("特征数据类型:\n", X.dtypes)  # 确保所有特征是数值型
# print("标签分布:\n", y.value_counts())  # 检查类别平衡性




In [21]:
help(MLPClassifier)

Help on class MLPClassifier in module sklearn.neural_network._multilayer_perceptron:

class MLPClassifier(sklearn.base.ClassifierMixin, BaseMultilayerPerceptron)
 |  MLPClassifier(hidden_layer_sizes=(100,), activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)
 |  
 |  Multi-layer Perceptron classifier.
 |  
 |  This model optimizes the log-loss function using LBFGS or stochastic
 |  gradient descent.
 |  
 |  .. versionadded:: 0.18
 |  
 |  Parameters
 |  ----------
 |  hidden_layer_sizes : array-like of shape(n_layers - 2,), default=(100,)
 |      The ith element represents the number of neurons in the ith
 |      hidden layer.
 |  
 |  activation : {'id

In [5]:
# split the training data and testing data, then do standarization
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, shuffle=True)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# construct the multi-layer perceptron model
# alpha: regularization parameter, solver: optimization algorithm, tol: stop condition
mlp_clf = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=100, alpha = 1e-4,
                        solver='lbfgs', tol=1e-6, random_state=42)

mlp_clf.fit(X_train_scaled, y_train)

y_train_pred = mlp_clf.predict(X_train_scaled)
y_test_pred = mlp_clf.predict(X_test_scaled)

C:\Users\14198\anaconda3\envs\Yingwen\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [7]:
accuracy_train = accuracy_score(y_train_pred, y_train)
accuracy_test = accuracy_score(y_test_pred, y_test)

print(f"Training accuracy: {accuracy_train:.4f}")
print(f"Testing accuracy: {accuracy_test:.4f}")

# Accuracy score
#Training accuracy: 0.9413
#Testing accuracy: 0.9302

Training accuracy: 0.9413
Testing accuracy: 0.9302


In [20]:
from sklearn.metrics import confusion_matrix, classification_report
from tabulate import tabulate

model_evaluation_metrics1 = {}

conf_matrix_train = confusion_matrix(y_train, y_train_pred)
classif_report_train = classification_report(y_train, y_train_pred)

model_evaluation_metrics1 = {
    'Confusion Matrix': conf_matrix_train,
    'Classification Report': classif_report_train
}

print(f"MLP Evaluation Metrics:")
print("Confusion Matrix:")
print(tabulate(conf_matrix_train, headers=mlp_clf.classes_, tablefmt='grid'))
print("\nClassification Report:")
print(classif_report_train)

model_evaluation_metrics2 = {}

conf_matrix_test = confusion_matrix(y_test, y_test_pred)
classif_report_test = classification_report(y_test, y_test_pred)

model_evaluation_metrics2 = {
    'Confusion Matrix': conf_matrix_test,
    'Classification Report': classif_report_test
}

print(f"MLP Evaluation Metrics:")
print("Confusion Matrix:")
print(tabulate(conf_matrix_test, headers=mlp_clf.classes_, tablefmt='grid'))
print("\nClassification Report:")
print(classif_report_test)

MLP Evaluation Metrics:
Confusion Matrix:
+------+------+-----+------+------+------+------+
|    0 |    1 |   2 |    3 |    4 |    5 |    6 |
+======+======+=====+======+======+======+======+
| 1550 |    4 |   0 |    1 |    0 |   32 |   27 |
+------+------+-----+------+------+------+------+
|    4 | 1007 |   0 |   31 |    4 |   14 |    1 |
+------+------+-----+------+------+------+------+
|    0 |    0 | 405 |    0 |    0 |    0 |    0 |
+------+------+-----+------+------+------+------+
|    3 |   26 |   0 | 1262 |   13 |    9 |    0 |
+------+------+-----+------+------+------+------+
|    1 |    2 |   0 |   20 | 1452 |   32 |   13 |
+------+------+-----+------+------+------+------+
|   23 |    6 |   0 |    4 |   22 | 1863 |  182 |
+------+------+-----+------+------+------+------+
|   33 |    0 |   0 |    0 |    3 |  144 | 2695 |
+------+------+-----+------+------+------+------+

Classification Report:
              precision    recall  f1-score   support

           0       0.96      

接下来可以尝试
1. 增加1，5，6类（错误率较高）的数据
2. 交叉验证调参

In [24]:
# First try cross-validation 
from sklearn.model_selection import GridSearchCV, StratifiedKFold

mlp_clf2 = MLPClassifier(early_stopping=True, random_state=42)

param_grid = {
    'hidden_layer_sizes':[(50,), (50, 20), (100,)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'lbfgs'],
    'alpha': [0.0001, 0.001],
    'max_iter': [200, 500],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(
    estimator=mlp_clf2,
    param_grid=param_grid,
    cv=cv,
    scoring='accuracy',  # 可根据需求改为 'f1_macro' 等
    n_jobs=-1,           # 并行计算
    verbose=1            # 打印进度
)

# 在标准化后的训练集上搜索
grid.fit(X_train_scaled, y_train)

# 输出最佳参数和交叉验证分数
print("Best parameters:", grid.best_params_)
print("Best CV accuracy:", grid.best_score_)

# 在测试集上评估
best_mlp = grid.best_estimator_
test_score = best_mlp.score(X_test_scaled, y_test)
print("Test set accuracy:", test_score)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 20), 'max_iter': 200, 'solver': 'lbfgs'}
Best CV accuracy: 0.9294638032726311
Test set accuracy: 0.9327947117150202


C:\Users\14198\anaconda3\envs\yingwen\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [25]:
y_test_pred = best_mlp.predict(X_test_scaled)

model_evaluation_metrics3 = {}

conf_matrix_test = confusion_matrix(y_test, y_test_pred)
classif_report_test = classification_report(y_test, y_test_pred)

model_evaluation_metrics3 = {
    'Confusion Matrix': conf_matrix_test,
    'Classification Report': classif_report_test
}

print(f"MLP Evaluation Metrics:")
print("Confusion Matrix:")
print(tabulate(conf_matrix_test, headers=mlp_clf.classes_, tablefmt='grid'))
print("\nClassification Report:")
print(classif_report_test)

MLP Evaluation Metrics:
Confusion Matrix:
+-----+-----+-----+-----+-----+-----+-----+
|   0 |   1 |   2 |   3 |   4 |   5 |   6 |
+=====+=====+=====+=====+=====+=====+=====+
| 392 |   2 |   0 |   1 |   0 |   8 |  10 |
+-----+-----+-----+-----+-----+-----+-----+
|   1 | 242 |   0 |  11 |   0 |   7 |   0 |
+-----+-----+-----+-----+-----+-----+-----+
|   0 |   0 | 117 |   0 |   0 |   0 |   0 |
+-----+-----+-----+-----+-----+-----+-----+
|   1 |  11 |   0 | 300 |   4 |   1 |   0 |
+-----+-----+-----+-----+-----+-----+-----+
|   0 |   4 |   0 |   5 | 386 |   8 |   5 |
+-----+-----+-----+-----+-----+-----+-----+
|   4 |   1 |   0 |   0 |   6 | 479 |  46 |
+-----+-----+-----+-----+-----+-----+-----+
|   5 |   0 |   0 |   0 |   1 |  41 | 624 |
+-----+-----+-----+-----+-----+-----+-----+

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       413
           1       0.93      0.93      0.93       261
           2       1.00